# Control Reachy's gaze in Unity using the look_at function

This notebook shows you how to control Reachy's head orientation by using the look_at function.

The look_at function takes a few argument:
* (x, y, z) the 3D coordinates of the point you want to look at (in Reachy's coordinate system : x is forward, y is left and z s up)
* duration: head motion duration to go from its current orientation to the requested one (in sec.)
* wait (bool): whether or not to wait for the end of the movement

Below, we illustrate how to use it inside a loop to let you define new look target using slider widgets.

In [1]:
from reachy import parts, Reachy

reachy = Reachy(
    left_arm=parts.LeftArm(io='ws', hand='force_gripper'),
    right_arm=parts.RightArm(io='ws', hand='force_gripper'),
    head=parts.Head(io='ws'),    
)

###Arm - init - io:  ws
###ReachyPart - init - name:  left_arm
###WsIO - shared_server - part_name:  left_arm
###part_name:  left_arm
###WsIO - shared_server - part_name:  left_arm  creating new ws
###attach_dxl_motors - name:  left_arm
###ReachyPart - init - name:  left_arm.hand
###WsIO - shared_server - part_name:  left_arm.hand
###part_name:  left_arm.hand
###attach_dxl_motors - name:  left_arm.hand
###Arm - init - io:  ws
###ReachyPart - init - name:  right_arm
###WsIO - shared_server - part_name:  right_arm
###part_name:  right_arm
###attach_dxl_motors - name:  right_arm
###ReachyPart - init - name:  right_arm.hand
###WsIO - shared_server - part_name:  right_arm.hand
###part_name:  right_arm.hand
###attach_dxl_motors - name:  right_arm.hand
###Head - init - io:  ws
###ReachyPart - init - name:  head
###WsIO - shared_server - part_name:  head
###part_name:  head
###Head.orbita_config:  {'Pc_z': [0, 0, 23], 'Cp_z': [0, 0, 0], 'R': 35.9, 'R0': array([[ 0.49240388, -0.8660254 ,  0.086

In [ ]:
reachy.head.io.motors[0].name

In [ ]:
reachy.right_arm.io.motors[0].name

In [ ]:
dir(reachy.head.io)

In [ ]:
reachy.right_arm.io

You can now connect your robot in Unity.

In [ ]:
from IPython.display import clear_output


class LookAt(object):
    def __init__(self, head, init_target=(0.5, 0, 0), period=0.05):
        self.head = head
        
        self.trajs = None
        
        self.target = init_target
        self.period = period
        
    def set_target(self, x, y, z):
        if self.target == (x, y, z):
            return
        
        if self.trajs is not None:
            for t in self.trajs:
                t.stop()
                
        self.target = (x, y, z)
        try:
            self.trajs = self.head.look_at(x, y, z, duration=self.period, wait=False)
        except (ValueError, ZeroDivisionError):
            clear_output(wait=True)
            print(f'target {self.target} out of reach!')

In [ ]:
look_at = LookAt(reachy.head)

In [ ]:
from ipywidgets import interact

def loop(x, y, z):
    look_at.set_target(x, y, z)
    
_ = interact(
    loop,
    x=0.5,
    y=(-1, 1, 0.01),
    z=(-1, 1, 0.01),
)

In [ ]:
for m in reachy.right_arm.motors:
    m.compliant = False

In [ ]:
reachy.goto({
    'right_arm.shoulder_pitch': 0,
    'right_arm.shoulder_roll': 0,
    'right_arm.arm_yaw': 0,    
    'right_arm.elbow_pitch': -90,
    'right_arm.hand.forearm_yaw': 0,
    'right_arm.hand.wrist_pitch': 0,
    'right_arm.hand.wrist_roll': 0,
    'right_arm.hand.gripper': 0,
}, duration=3, wait=True)

In [ ]:
dir(reachy.right_arm.motors[0])

In [ ]:
reachy.right_arm.motors[0].offset